# 1. load/reload all dependency

In [4]:
import numpy as np
import pandas as pd
from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from MedCode import CodeMapping

import pyhealth.datasets as datasets
import pyhealth.models as models
import pyhealth.utils as utils
from importlib import reload
reload(datasets)
reload(models)
reload(utils)

import torch
print(torch.cuda.is_available())


ModuleNotFoundError: No module named 'pandas'

# 2. get MIMIC-III dataset object

In [2]:
# choose a target code: ATC4, NDC or RXCUI
target_code = 'ATC4'
tool = CodeMapping('RxNorm', target_code)
tool.load_mapping()

# load dataset by passing arguments
dataset = datasets.MIMIC_III(
    table_names=['med', 'prod', 'diag'], 
    code_map=tool.RxNorm_to_ATC4,
)

----- preparing code mappings -----
source 1 loaded from https://drive.google.com/uc?id=1I2G6fsBDXDiAK95qFWwtnl3Ib2MaLeCx
source 2 loaded from https://drive.google.com/uc?id=1d2HzsByXrPadvjaKDOEaOt78OkAZOrjC
source 3 loaded from https://drive.google.com/uc?id=199i8mP2gMQNhwUe-2ZNmIr5fhiBbzVlK
source 4 loaded from https://drive.google.com/uc?id=1Z11J4st1sI44jPborls9jIxzcpF-GpGt
mapping finished: RxNorm -> ATC4
mapping finished: ATC4 -> RxNorm
load time: 6.7135419845581055s
-----------------------------------------
loaded the med table!
loaded the prod table!
loaded the diag table!
generated .pat_to_visit!
generated .visit_dict!
generated .encoded_visit_dict!
generated .maps (for code to index mappings)!


# 3. model train and test

In [4]:
# initialize RETAIN model and build the dataloaders
model = models.RETAIN(
    voc_size=dataset.voc_size,
    emb_size=64,
)
dataset.get_dataloader("RETAIN")

# set checkpoint and trainer
checkpoint_callback = ModelCheckpoint(dirpath='./model_cpt')
trainer = Trainer(
    gpus=1,
    max_epochs=20,
    progress_bar_refresh_rate=5,
    callbacks=[checkpoint_callback],
)

# train model
trainer.fit(
    model=model,
    train_dataloaders=dataset.train_loader,
    val_dataloaders=dataset.val_loader,
)

# load the best model for test
checkpoint_callback.best_model_path
model.summary(dataset.test_loader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


generated train/val/test dataloaders for RETAIN model!


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]

  | Name      | Type       | Params
-----------------------------------------
0 | embedding | Sequential | 558 K 
1 | alpha_gru | GRU        | 25.0 K
2 | beta_gru  | GRU        | 25.0 K
3 | alpha_li  | Linear     | 65    
4 | beta_li   | Linear     | 4.2 K 
5 | output    | Linear     | 7.9 K 
-----------------------------------------
621 K     Trainable params
0         Non-trainable params
621 K     Total params
2.484     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]


Jaccard: 0.4003,  PRAUC: 0.7004, AVG_PRC: 0.5304, AVG_RECALL: 0.6836, AVG_F1: 0.5646, AVG_MED: 31.13

